In [109]:
import pandas as pd

CPI_EUA = pd.read_csv("data/CPI eua.csv")
FEDERAL_FUND_EFFECTVE = pd.read_csv("data/Federal Funds Effective.csv")
INFLACA_EUA= pd.read_csv('data/Inflação eua.csv')
# IPCA = pd.read_csv('data/IPCA.csv', sep=";", encoding="iso-8859-1")
# JPY_USD = pd.read_csv('data/JPY_USD Dados Históricos.csv')
PIB_BR = pd.read_excel('data/PIB Br.xlsx', index_col=[0])
PIB_EUA = pd.read_csv('data/PIB eua.csv')
RELACAO_EURO_DOLAR = pd.read_csv('data/Relação euro_dolar.csv', sep=";", encoding="iso-8859-1")
RESERVAR_INTERNACIONAIS = pd.read_csv('data/reservas internacionais.csv', sep=";", encoding="iso-8859-1")
RISCO_BR = pd.read_excel('data/Risco Br.xlsx')
# SALDO_BALANCA_COMERCIAL = pd.read_csv('data/Saldo da Balanca Comercial.csv', sep=";", encoding="iso-8859-1")
TAXA_CAMBIO_REAL_EFETIVA = pd.read_csv('data/Taxa Cambio Real Efetiva.csv', sep=";", encoding="iso-8859-1")
TAXA_SELIC = pd.read_csv('data/Taxa Selic.csv', sep=";")
USB_BRL_HIST = pd.read_csv('data/USD_BRL Dados Históricos.csv')

In [95]:
CPI_EUA["ts"] = pd.to_datetime(CPI_EUA["DATE"])
FEDERAL_FUND_EFFECTVE["ts"] = pd.to_datetime(FEDERAL_FUND_EFFECTVE["DATE"])
INFLACA_EUA["ts"] = pd.to_datetime(INFLACA_EUA["DATE"])
# PIB_BR["ts"] = pd.to_datetime(PIB_BR["Unnamed: 1"], format=") TODO: arrumar
PIB_EUA["ts"] = pd.to_datetime(PIB_EUA["DATE"])
RELACAO_EURO_DOLAR["ts"] = pd.to_datetime(RELACAO_EURO_DOLAR["Data"], format="%m/%Y", errors="coerce")
RESERVAR_INTERNACIONAIS["ts"] = pd.to_datetime(RESERVAR_INTERNACIONAIS["Data"], format="%m/%Y", errors="coerce")
RISCO_BR["ts"] = pd.to_datetime(RISCO_BR["Mês"], format="%b-%y", errors="coerce")
TAXA_CAMBIO_REAL_EFETIVA["ts"] = pd.to_datetime(TAXA_CAMBIO_REAL_EFETIVA["Data"], format="%m/%Y", errors="coerce")
TAXA_SELIC["ts"] = pd.to_datetime(TAXA_SELIC["Data"], format="%d/%m/%Y", errors="coerce")
USB_BRL_HIST["ts"] = pd.to_datetime(USB_BRL_HIST["Data"], format="%d.%m.%Y", errors="coerce")

In [112]:


dataframes = [CPI_EUA, FEDERAL_FUND_EFFECTVE, INFLACA_EUA, PIB_EUA, RELACAO_EURO_DOLAR, RESERVAR_INTERNACIONAIS, RISCO_BR, TAXA_CAMBIO_REAL_EFETIVA, USB_BRL_HIST]
functools.reduce(lambda left, right: pd.merge(left, right, on="ts", how="inner"), dataframes)

NameError: name 'functools' is not defined